In [1]:
import openfermion
import openfermionpyscf
from openfermion import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.transforms import get_fermion_operator
from openfermion.circuits import (uccsd_singlet_get_packed_amplitudes,
                               uccsd_singlet_generator, uccsd_generator,
                               uccsd_convert_amplitude_format)
import numpy as np
import cs_vqe as c
import itertools
import qubit_conversion as q_conv

In [70]:
import ast
import matplotlib
import matplotlib.pyplot as plt

f = open("hamiltonians.txt","r")
hamiltonians = ast.literal_eval(f.read())
f.close()

for h in hamiltonians.keys():
    print(h, '# qubits:', hamiltonians[h][1], 'A terms:', hamiltonians[h][5][3][1])

H2-S1_STO-3G_singlet # qubits: 18 A terms: ['IIZZZZZZZXIIIIIIII', 'IIIIIIIIIZIIIIIIII']
C1-O1_STO-3G_singlet # qubits: 16 A terms: ['IIZZZZZXIIIIIIII', 'IIIIIIIZIIIIIIII']
H1-Cl1_STO-3G_singlet # qubits: 16 A terms: ['IIZZZZZXIIIIIIII', 'IIIIIIIZIIIIIIII']
H1-Na1_STO-3G_singlet # qubits: 16 A terms: ['IIZZZZZZZXIIIIII', 'IIIIIIIIIZIIIIII']
H2-Mg1_STO-3G_singlet # qubits: 17 A terms: ['IIIZZIIZXZZIIZZZZ', 'IIIIIIIIZIIIIIIII']
H1-F1_3-21G_singlet # qubits: 18 A terms: ['IIIIZZZZZXZZIIIIII', 'IIZIIIIIZZIIIIIIII']
H1-Li1_3-21G_singlet # qubits: 18 A terms: ['IIIIZZZZIIZXIIIIII', 'IIZIIIIIIIZZIIIIII']
Be1_STO-3G_singlet # qubits: 5 A terms: ['IIIXI', 'IIIZI']
H1-F1_STO-3G_singlet # qubits: 8 A terms: ['IIXZZZZZ', 'IIZIIIII']
H1-Li1_STO-3G_singlet # qubits: 8 A terms: ['IIIIIXZZ', 'ZZIZIZIZ']
Ar1_STO-3G_singlet # qubits: 13 A terms: ['IIIZZZZZXZZZZ', 'IIIIIIIIZIIII']
F2_STO-3G_singlet # qubits: 15 A terms: ['IIIIIIIIIIIIZII', 'IIIIIIIIIIIXXYY']
H1-O1_STO-3G_singlet # qubits: 8 A terms: ['IIX

In [94]:
speciesname = 'H2-O1_STO-3G_singlet'

encoding = hamiltonians[speciesname][0] # in this dataset, always 'JW' for Jordan-Wigner, but leaves room for trying Bravyi-Kitaev as well
num_qubits = hamiltonians[speciesname][1] # number of qubits (all of these Hamiltonians have been tapered for molecular symmetries)
ham = hamiltonians[speciesname][2] # full Hamiltonian
ham_noncon = hamiltonians[speciesname][3] # noncontextual part of Hamiltonian, found by greedy DFS
true_gs = hamiltonians[speciesname][4] # ground state energy of full Hamiltonian (in Hartree)
gs_noncon = hamiltonians[speciesname][5] # list containing information about noncontextual ground state: zeroth entry is ground state energy of noncontextual part of Hamiltonian
#print('commuting generators:',model[0], '\n')
#print('anticommuting generators:',model[1], '\n')
#print('term reconstruction:',model[2], '\n')

print('number of qubits:', num_qubits)
print('true ground state energy:', true_gs)
print('noncontextual approximation to ground state energy:', gs_noncon[0])
print(ham_noncon)

number of qubits: 10
true ground state energy: -83.92870248174707
noncontextual approximation to ground state energy: -83.87422390061549
{'IIIIIIIIII': -55.226577416214326, 'IIIIIIIIIZ': 12.411397933347452, 'IIIIIIIIZI': 12.411397933347452, 'IZIIIIIIII': 2.73330884423689, 'IIIIIIIZII': 1.6496131029811243, 'IIIIIIZIII': 1.6496131029811243, 'IIIZIIIIII': 1.2974887568152718, 'IIZIIIIIII': 1.2974887568152718, 'IIIIIZIIII': 1.1909223847985508, 'IIIIZIIIII': 1.1909223847985508, 'IIIIIIIIZZ': 1.1862777224861802, 'ZIIIIIIIII': 0.8108294833868595, 'ZIIIZZIIII': 0.8108294833868593, 'ZZZIZIZIZI': -0.7928440928467336, 'ZZIZZIIZIZ': -0.7928440928467335, 'IZIIIIIIIZ': 0.5511652977306052, 'IZIIIIIIZI': 0.5511652977306052, 'IIIIZXZZZZ': -0.3691171177897613, 'ZIIIIXZZZZ': 0.3691171177897613, 'IZIZIIIIII': 0.3454257236707442, 'IZZIIIIIII': 0.3454257236707442, 'IZIIIIIZII': 0.3377138999112341, 'IZIIIIZIII': 0.3377138999112341, 'IIIIZZIIII': 0.3096300921948822, 'IZIIIZIIII': 0.30522458512105066, 'IZIIZIII

In [4]:
import openfermion
import openfermionpyscf
from openfermion import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.transforms import get_fermion_operator
from openfermion.circuits import (uccsd_singlet_get_packed_amplitudes,
                               uccsd_singlet_generator, uccsd_generator,
                               uccsd_convert_amplitude_format)
import numpy as np
import cs_vqe as c
import itertools
import qubit_conversion as q_conv

singlet_bool = True # Set general UCCSD or singlet UCCSD.

bond_len = 0.772#1.45
atom_1 = 'He'
atom_2 = 'H'
basis = '3-21g'
multiplicity = 1
charge = 1

coordinate_1 = (0.0, 0.0, 0.0)
coordinate_2 = (0.0, 0.0, bond_len)
geometry = [(atom_1, coordinate_1), (atom_2, coordinate_2)]

molecule_data = MolecularData(geometry, basis, multiplicity, charge, description='Test')
#molecule.load()

# Set calculation parameters.
run_scf = 1
run_mp2 = 1
run_cisd = 0
run_ccsd = 0
run_fci = 1
delete_input = True
delete_output = True

# Run pyscf.
molecule = run_pyscf(molecule_data,
                     run_scf=run_scf,
                     run_mp2=run_mp2,
                     run_cisd=run_cisd,
                     run_ccsd=run_ccsd,
                     run_fci=run_fci)

#molecule.load()
#print(molecule)

ham = get_fermion_operator(molecule.get_molecular_hamiltonian())
ham_q = jordan_wigner(ham)
print('Hamiltonian:', '\n', ham_q, '\n')

scf = True      # Hartree-Fock.
mp2 = True      # Moller-Plesset 2.
cisd = True     # Configuration interaction singles and doubles.
ccsd = True     # Coupled cluster singles and doubles.
fci = True      # Full configuration interaction.

calculated_molecule = run_pyscf(molecule_data, scf, mp2, cisd, ccsd, fci)

if ccsd:
    ccsd_single_amps = calculated_molecule.ccsd_single_amps
    ccsd_double_amps = calculated_molecule.ccsd_double_amps

num_electrons = calculated_molecule.n_electrons
num_qubits = 2*calculated_molecule.n_orbitals

if singlet_bool:
    # Get singlet UCCSD generator.
    packed_amps = uccsd_singlet_get_packed_amplitudes(ccsd_single_amps,  ccsd_double_amps, num_qubits, num_electrons)
    ucc_sing = uccsd_singlet_generator(packed_amps, num_qubits, num_electrons)
    #print(ucc_sing)

else:
    # Get general UCCSD operator.
    ucc_op = uccsd_generator(ccsd_single_amps, ccsd_double_amps)
    #print(ucc_op)
    
ucc_q = jordan_wigner(ucc_sing)
print('UCCSD ansatz:', '\n', ucc_q)

Hamiltonian: 
 (1.1324434021069416+0j) [] +
(-0.025810254414815714+0j) [X0 X1 Y2 Y3] +
(-0.012799934786436765+0j) [X0 X1 Y2 Z3 Z4 Y5] +
(-0.025159645839898225+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Y7] +
(-0.012799934786436765+0j) [X0 X1 X3 X4] +
(-0.025159645839898225+0j) [X0 X1 X3 Z4 Z5 X6] +
(-0.01711452882177453+0j) [X0 X1 Y4 Y5] +
(-0.0009891049103622836+0j) [X0 X1 Y4 Z5 Z6 Y7] +
(-0.0009891049103622836+0j) [X0 X1 X5 X6] +
(-0.05469856574684896+0j) [X0 X1 Y6 Y7] +
(0.025810254414815714+0j) [X0 Y1 Y2 X3] +
(0.012799934786436765+0j) [X0 Y1 Y2 Z3 Z4 X5] +
(0.025159645839898225+0j) [X0 Y1 Y2 Z3 Z4 Z5 Z6 X7] +
(-0.012799934786436765+0j) [X0 Y1 Y3 X4] +
(-0.025159645839898225+0j) [X0 Y1 Y3 Z4 Z5 X6] +
(0.01711452882177453+0j) [X0 Y1 Y4 X5] +
(0.0009891049103622836+0j) [X0 Y1 Y4 Z5 Z6 X7] +
(-0.0009891049103622836+0j) [X0 Y1 Y5 X6] +
(0.05469856574684896+0j) [X0 Y1 Y6 X7] +
(0.0025779051794141486+0j) [X0 Z1 X2] +
(0.004766148105680677+0j) [X0 Z1 X2 X3 Z4 X5] +
(-0.009264702923308317+0j) [X0 Z1 X2 X3 

UCCSD ansatz: 
 0.006478936130102838j [X0 X1 X2 Y3] +
0.006478936130102838j [X0 X1 Y2 X3] +
0.0020562744029710704j [X0 X1 X4 Y5] +
0.0020562744029710704j [X0 X1 Y4 X5] +
0.005083283191416048j [X0 X1 X6 Y7] +
0.005083283191416048j [X0 X1 Y6 X7] +
-0.006478936130102838j [X0 Y1 X2 X3] +
0.006478936130102838j [X0 Y1 Y2 Y3] +
-0.0020562744029710704j [X0 Y1 X4 X5] +
0.0020562744029710704j [X0 Y1 Y4 Y5] +
-0.005083283191416048j [X0 Y1 X6 X7] +
0.005083283191416048j [X0 Y1 Y6 Y7] +
-0.006360390047729467j [X0 Z1 Y2] +
-0.002645411873769119j [X0 Z1 Z2 Z3 Y4] +
0.0009221551709274561j [X0 Z1 Z2 Z3 Z4 Z5 Y6] +
-0.006478936130102838j [Y0 X1 X2 X3] +
0.006478936130102838j [Y0 X1 Y2 Y3] +
-0.0020562744029710704j [Y0 X1 X4 X5] +
0.0020562744029710704j [Y0 X1 Y4 Y5] +
-0.005083283191416048j [Y0 X1 X6 X7] +
0.005083283191416048j [Y0 X1 Y6 Y7] +
-0.006478936130102838j [Y0 Y1 X2 Y3] +
-0.006478936130102838j [Y0 Y1 Y2 X3] +
-0.0020562744029710704j [Y0 Y1 X4 Y5] +
-0.0020562744029710704j [Y0 Y1 Y4 X5] +
-0.0

In [4]:
ham = q_conv.QubitOperator_to_dict(ham_q, num_qubits)
anz_terms = list((q_conv.QubitOperator_to_dict(ucc_q, num_qubits)).keys())
terms_noncon = c.greedy_dfs(ham, 1, criterion='weight')[-1]
ham_noncon = {t:ham[t] for t in terms_noncon}
#ham_noncon
terms_context = list(ham.keys() - terms_noncon)
ham_context = {t:ham[t] for t in terms_context}
#ham_context
c.contextualQ_ham(ham_context)

NameError: name 'ham_q' is not defined

In [5]:
from qiskit.aqua.algorithms import NumPyEigensolver

result = NumPyEigensolver(q_conv.dict_to_WeightedPauliOperator(ham)).run()
true_gs = np.real(result.eigenvalues)

print(true_gs)

/home/tim/anaconda3/lib/python3.7/site-packages/qiskit/aqua/algorithms/eigen_solvers/eigen_solver.py:38: DeprecationWarning: The package qiskit.aqua.algorithms.eigen_solvers is deprecated. It was moved/refactored to qiskit.algorithms.eigen_solvers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  'qiskit-terra')


[-8.93327607]


/home/tim/anaconda3/lib/python3.7/site-packages/qiskit/aqua/operators/operator_base.py:46: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('aqua.operators', 'qiskit.opflow', 'qiskit-terra')


In [95]:
model = c.quasi_model(ham_noncon)
fn_form = c.energy_function_form(ham_noncon, model)
gs_noncon = c.find_gs_noncon(ham_noncon)
gs_noncon_energy = gs_noncon[0]
ep_state = gs_noncon[1]
ham_context = {p:c for p,c in ham.items() if p not in ham_noncon}
c.csvqe_approximations_heuristic(ham, ham_noncon, num_qubits, true_gs)
#c.contextualQ_ham(ham_context)

[-83.92870248174707,
 [-83.8742239006154,
  -83.8775347638022,
  -83.89065311104831,
  -83.89785356786018,
  -83.91205889216387,
  -83.91813131301454,
  -83.92761703951169,
  -83.92862837750931,
  -83.92865354102884,
  -83.92870248174634,
  -83.92870248174634],
 [0.054478581131661485,
  0.051167717944863966,
  0.03804937069875791,
  0.030848913886885043,
  0.016643589583196672,
  0.010571168732525393,
  0.001085442235378764,
  7.410423775411346e-05,
  4.8940718230028324e-05,
  7.247535904753022e-13,
  7.247535904753022e-13],
 [4, 0, 3, 2, 5, 6, 1, 8, 7, 9]]

In [15]:
model[0:2], gs_noncon_energy, ep_state

((['ZIIIIIIIII',
   'IZIIIIIIII',
   'IIZIIIIIII',
   'IIIZIIIIII',
   'IIIIZIIIII',
   'IIIIIIZIII',
   'IIIIIIIZII',
   'IIIIIIIIZI',
   'IIIIIIIIIZ'],
  ['IIIIZXZZZZ', 'IIIIIZIIII']),
 -83.87422390061549,
 [[1, -1, -1, -1, -1, -1, -1, -1, -1],
  [9.576858245891391e-08, -0.9999999999999954]])

# The epistemic state defines a classical probability distribution:

In [16]:
def ontic_prob(ep_state, ontic_state):
    
    if ep_state[0] != ontic_state[0]:
        return 0
    
    else:
        prod = 1
        for index, r in enumerate(ep_state[1]):
            f = 1/2 * abs(r + ontic_state[1][index])
            prod *= f
        
        return prod    

def epistemic_dist(ep_state):
    size_G = len(ep_state[0])
    size_Ci = len(ep_state[1])
    size_R = size_G + size_Ci
    
    ep_prob = {}
    
    ontic_states = list(itertools.product([1, -1], repeat=size_R))
    
    for o in ontic_states:
        o_state = [list(o[0:size_G]), list(o[size_G:size_R])]
        o_prob = ontic_prob(ep_state, o_state)
        
        if o_prob != 0:
            ep_prob[o] = o_prob
    
    return ep_prob

In [17]:
epistemic_dist(ep_state)

{(1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1): 1.1379787092233828e-15,
 (1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1): 0.5000000478842901,
 (1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1): 1.137978491258188e-15,
 (1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1): 0.49999995211570764}

# Find a simultaneous eigenstate of the operators G_i and A:

In [18]:
from copy import deepcopy

G = model[0]
fixed_qubits = ['I' for i in range(num_qubits)]
basis_ops = []
mult_Z_indices = []

for index, g in enumerate(G):
    Z_indices = [i for i, p in enumerate(list(g)) if p == 'Z']
    if len(Z_indices) == 1:
        if ep_state[0][index] == -1:
            fixed_qubits[Z_indices[0]] = 1
        else:
            fixed_qubits[Z_indices[0]] = 0
    else:
        mult_Z_indices.append([ep_state[0][index], Z_indices])
        

for Z_indices in mult_Z_indices:
    Z0, Z1 = Z_indices[1][0], Z_indices[1][1] 
    q0, q1 = fixed_qubits[Z0], fixed_qubits[Z1]
    
    if q0 != q1:
        if q0 != 'I':
            if Z_indices[0] == -1:
                fixed_qubits[Z1] = (q0+1)%2
            else:
                fixed_qubits[Z1] = q0
        else:
            if Z_indices[0] == -1:
                fixed_qubits[Z0] = (q1+1)%2
            else:
                fixed_qubits[Z0] = q1
        mult_Z_indices.remove(Z_indices)
            
for Z_indices in mult_Z_indices:
    if Z_indices[0] == -1:
        fixed = deepcopy(fixed_qubits)
        fixed[Z_indices[1][0]] = 0
        fixed[Z_indices[1][1]] = 1
        fixed = [str(i) for i in fixed]
        basis_ops.append(''.join(fixed))
        
        fixed = deepcopy(fixed_qubits)
        fixed[Z_indices[1][0]] = 1
        fixed[Z_indices[1][1]] = 0
        fixed = [str(i) for i in fixed]
        basis_ops.append(''.join(fixed))
    
print(basis_ops)

[]


In [88]:
r1 = ep_state[1][0]
r2 = ep_state[1][1]
t = np.arctan((r2 + 1) / r1)
t

-4.753035164654311e-08

In [91]:
def bin_to_int(bin_str):
    bits = [int(b) for b in bin_str]
    for index, b in enumerate(bits):
        bits[index] = b * 2 ** (len(bits)-index-1)
    return sum(bits)

psi = [0 for i in range(2**num_qubits)]
psi[bin_to_int('0111101111')] = np.sin(t)
psi[bin_to_int('0111111111')] = np.cos(t)
#print(psi)

In [90]:
from openfermion.linalg import LinearQubitOperator

psi = np.array(psi)
ham_q = q_conv.dict_to_QubitOperator(ham, num_qubits)
ham_noncon_q = q_conv.dict_to_QubitOperator(ham_noncon, num_qubits)
ham_context_q = q_conv.dict_to_QubitOperator(ham_context, num_qubits)

#take expectation value algebraically
H = LinearQubitOperator(ham_noncon_q, num_qubits)
H_psi = H.matvec(psi)
expect = psi.dot(H_psi)

print('<H> w.r.t. simultaneous eigenstate:', expect)
print('Noncontextual approximation:', gs_noncon_energy)
print('Expectation values match?', abs(expect - gs_noncon_energy) < 10**-12) #sanity check - expectation values match? 

<H> w.r.t. simultaneous eigenstate: (-83.87422390061543+0j)
Noncontextual approximation: -83.87422390061549
Expectation values match? True


# Do we have any quantum corrections?

In [52]:
G = model[0]
terms_context = list(ham_context.keys())
G

['ZIIIIIIIII',
 'IZIIIIIIII',
 'IIZIIIIIII',
 'IIIZIIIIII',
 'IIIIZIIIII',
 'IIIIIIZIII',
 'IIIIIIIZII',
 'IIIIIIIIZI',
 'IIIIIIIIIZ']

In [53]:
q_corr_terms = []
for t in terms_context:
    commutes = []
    for g in G:
        if c.commute(t, g):
            commutes.append(g)
    #print(t, 'commutes with the noncontextual generators:', commutes)
    if commutes == G:
        q_corr_terms.append(t)
        
if q_corr_terms == []:
    print('No quantum correction')
else:
    print('Terms admitting quantum correction:', q_corr_terms)

No quantum correction


In [54]:
#psi2 = [0 for i in range(2**num_qubits)]
#psi2[bin_to_int('000011')] = np.sin(t)
#psi2[bin_to_int('010111')] = -np.cos(t)

In [55]:
#from qiskit import QuantumCircuit
#from qiskit.extensions import Initialize
#from qiskit.circuit import Parameter

#anz = QuantumCircuit(num_qubits) # We are redefining qc
#anz.initialize(psi)
#anz.ry(Parameter('x'), 3)
#anz.initialize(psi2)
#anz.rx(Parameter('y'), 3)

In [56]:
#from qiskit.utils import QuantumInstance, algorithm_globals
#from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
#from qiskit.algorithms import VQE
#from qiskit.quantum_info.operators.symplectic.pauli import Pauli
#from qiskit.opflow.primitive_ops import PauliOp
#from qiskit import Aer

#ham_qiskit = sum([PauliOp(Pauli(k), ham[k]) for k in ham.keys()])

#seed = 50
#algorithm_globals.random_seed = seed
#qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)
#slsqp = SLSQP(maxiter=1000)

#vqe = VQE(anz, optimizer=slsqp, quantum_instance=qi)
#vqe_run    = vqe.compute_minimum_eigenvalue(operator=ham_qiskit)
#vqe_result = vqe_run.optimal_value# + shift

#print('VQE:', vqe_result, '|', 'Noncontextual Ground State:', gs_noncon_energy, 'True Ground State:', true_gs)

In [57]:
from qiskit.aqua.algorithms import NumPyEigensolver

order = [0,1,2,3,4,5,6,7]
print(c.contextual_subspace_approximations(ham,model,fn_form,ep_state,order))

result = NumPyEigensolver(q_conv.dict_to_WeightedPauliOperator(ham)).run()
exact_energy = np.real(result.eigenvalues)

print(exact_energy)

[-83.8742239006154, -83.87422390061546, -83.8758195779362, -83.88240004021091, -83.89086618279309, -83.91323713466015, -83.91937617916174, -83.92862837750964, -83.92865354102884]
[-83.92870248]


In [58]:
rotations = (c.diagonalize_epistemic(model,fn_form,ep_state))[0]
rotations

[]

In [59]:
from copy import deepcopy

rot_psi = deepcopy(psi)

for r in rotations:
    r_op = LinearQubitOperator(QubitOperator('', 1/np.sqrt(2)) + q_conv.dict_to_QubitOperator({r[1]: -1/np.sqrt(2)*1j}, num_qubits), num_qubits)
    rot_psi = r_op.matvec(rot_psi)
    
#print(rot_psi)

In [60]:
from openfermion.linalg import LinearQubitOperator

rot_psi = np.array(rot_psi)
rot_ham = c.get_reduced_hamiltonians(ham,model,fn_form,ep_state,order=[0,1,2,3,4,5,6,7])[8]
rot_ham_q = q_conv.dict_to_QubitOperator(rot_ham, num_qubits)

#take expectation value algebraically
rot_H = LinearQubitOperator(rot_ham_q, num_qubits)
rot_H_psi = rot_H.matvec(rot_psi)
rot_expect = rot_psi.dot(rot_H_psi)

print('<H> w.r.t. simultaneous eigenstate:', rot_expect)
print('Noncontextual approximation:', gs_noncon_energy)
print('Expectation values match?', abs(rot_expect - gs_noncon_energy) < 10**-14) #sanity check - expectation values match? 

<H> w.r.t. simultaneous eigenstate: (-83.87422390061546+0j)
Noncontextual approximation: -83.87422390061549
Expectation values match? False


In [61]:
rot_psi.dot(rot_H.matvec(rot_psi)), psi.dot(H.matvec(psi))

((-83.87422390061546+0j), (-83.87422390061543+0j))

array([ True,  True,  True, ...,  True,  True,  True])